## Bibliotecas

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql.window import Window

## Setup

In [2]:
import findspark

findspark.init()
import pyspark

In [3]:
spark = SparkSession.builder.config('spark.executor.memory', '8G').getOrCreate()

In [15]:
cnae_path = './data/CNAE/'

# Leitura dos arquivos com schema

In [36]:
schema1 = '_c0 STRING, _c1 STRING'

In [63]:

df_cnae = (spark.read
    .format('csv')
    .option('sep', ';')
    .option('encoding', 'ISO-8859-1')
     .schema(schema1)
    .load(cnae_path + 'CNAE.csv')
)


In [64]:
df_cnae.limit(5).show()



+-------+--------------------+
|    _c0|                 _c1|
+-------+--------------------+
|0111301|    Cultivo de arroz|
|0111302|    Cultivo de milho|
|0111303|    Cultivo de trigo|
|0111399|Cultivo de outros...|
|0112101|Cultivo de algodã...|
+-------+--------------------+



In [65]:
df_cnae.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [71]:
#arquivo 2
schema2 = '_c0 STRING, _c1 STRING'
read_options = {
    'encoding': 'ISO-8859-1',
    'sep': ';',  
    'escape': "\"" ,
}

df_munic = (
    spark.read
    .format('csv')
    .options(**read_options)
    .schema(schema2)
    .load(cnae_path + 'F.K03200$Z.D10710.MUNIC.csv')
)

In [72]:
df_munic.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [68]:
df_munic.limit(5).show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
|0001|       GUAJARA-MIRIM|
|0002|ALTO ALEGRE DOS P...|
|0003|         PORTO VELHO|
|0004|             BURITIS|
|0005|           JI-PARANA|
+----+--------------------+



# Renomeando as colunas

In [74]:
(
    df_cnae
    .withColumnRenamed('_c0', 'codigo')
    .withColumnRenamed('_c1', 'atividade_economica')
    .show(5)
)

+-------+--------------------+
| codigo| atividade_economica|
+-------+--------------------+
|0111301|    Cultivo de arroz|
|0111302|    Cultivo de milho|
|0111303|    Cultivo de trigo|
|0111399|Cultivo de outros...|
|0112101|Cultivo de algodã...|
+-------+--------------------+
only showing top 5 rows



In [73]:
(
    df_munic
    .withColumnRenamed('_c0', 'codigo')
    .withColumnRenamed('_c1', 'nome_do_municipio')
    .show(5)
)

+------+--------------------+
|codigo|   nome_do_municipio|
+------+--------------------+
|  0001|       GUAJARA-MIRIM|
|  0002|ALTO ALEGRE DOS P...|
|  0003|         PORTO VELHO|
|  0004|             BURITIS|
|  0005|           JI-PARANA|
+------+--------------------+
only showing top 5 rows



### Colunas e Expressões

As colunas são a principal unidade de manipulação de dados do Spark. 

In [ ]:
from pyspark.sql.functions import col, round

(
    df_cnae.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn("runtimeHours", round(col('runTimeMinutes').cast('int') / 60, 3))
    .show(5)
)

Forma "pandas" de selecionar:

1. `df.coluna`
2. `df['coluna']`

In [ ]:
(
    df_titles.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn("runtimeHours", df_titles['runtimeMinutes'].cast('int') / 60 )
    .show(5)
)

In [ ]:
(
    df_titles.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn("runtimeHours", df_titles['runtimeMinutes'].cast('int') / 60 )
    .withColumn("hours_plus2", df_titles['runtimeHours'] + 2 )
    .show(5)
)

In [ ]:
(
    df_titles.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn("runtimeHours", col('runTimeMinutes').cast('int') / 60 )
    .withColumn("hours_plus2", col('runtimeHours') + 2 )
    .show(5)
)

#### Expressões

In [ ]:
from pyspark.sql.functions import expr

(
    df_titles.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn("runtimeHours", expr('round(cast(runTimeMinutes as INT) / 60, 3)') )
    .show(5)
)

### Seleção de Colunas

In [ ]:
df_titles.show(5)

In [ ]:
df_titles.columns

In [ ]:
df_titles.select('tconst', 'primaryTitle', 'genres').show(10)

In [ ]:
select_cols = [c for c in df_titles.columns if c.find('Title') != -1]

In [ ]:
cols = ['tconst', 'primaryTitle', 'genres']
df_titles.select(select_cols).show(10)

In [ ]:
cols = ['primaryTitle', 'genres']
df_titles.select('tconst', *cols).show(10)

In [ ]:
df_titles.select('*').show(10)

Observações:
* Podemos realizar operações sobre colunas selecionadas. 
* A ordem em que as colunas são selecionadas é a ordem em que elas vão ser inseridas no DataFrame resultante.

In [ ]:
from pyspark.sql.functions import upper, expr

df_titles.select('tconst', 'genres', expr('upper(primaryTitle) as primaryTitle')).show(10)

In [ ]:
df_titles.selectExpr('tconst', 'genres', 'upper(primaryTitle) as primaryTitle').show(10)
df_titles.limit(5).show()

#### Selecionando valores distintos

In [ ]:
df_titles.dropDuplicates(subset=['startYear']).show()

In [ ]:
df_titles.select('startYear').distinct().show()

In [ ]:
df_titles.dropDuplicates(subset=['startYear']).show()

In [ ]:
df_titles.count()

### Filtros

Operadores lógicos:
* e: &
* ou: |
* não: ~

Para fazer o filtro, pode ser utilizado tanto a função `filter()` como `where()`.

#### Filtros com uma condição

In [ ]:
(
    df_titles.filter(~(col('titleType') == 'movie'))
    .count()
)

In [ ]:
(
    df_titles.where(col('titleType') == 'movie')
    .show(5)
)

#### Filtros com duas ou mais condições
Cada uma das condições deve estar entre parênteses e separada por um operador lógico. Naturalmente, é possível também "aninhar" condições, seguindo essa mesma lógica

In [ ]:
(
    df_titles.filter((col('titleType') == 'movie') & (col('runtimeMinutes') <= 90))
    .show(5)
)

In [ ]:
(
    df_titles.filter((col('titleType') == 'movie') & (col('runtimeMinutes') <= 90))
    .count()
)

In [ ]:
(
    df_titles.filter(((col('titleType') == 'movie') | (col('titleType') == 'tvSeries')) & (col('runtimeMinutes') <= 90))
    .count()
)

In [ ]:
(
    df_titles.filter((col('titleType').isin('movie', 'tvSeries')) & (col('runtimeMinutes') <= 90))
    .count()
)

In [ ]:
(
    df_titles
    .filter(col('titleType').isin('movie','tvSeries'))
    .filter(col('runtimeMinutes') <= 90)
    .count()
)

#### Filtros Utilizando Expressões

In [ ]:
(
    df_titles
    .filter('titleType = "movie"')
    .show(5)
)

In [ ]:
(
    df_titles
    .filter('titleType in ("movie", "tvSeries") and runtimeMinutes <= 90')
    .show(5)
)

#### Observações
Quando nos referimos às colunas por meio da função `col()`, temos acesso à diversos métodos das colunas que podem ser utilizados para auxliar na filtragem do DataFrame. Alguns deles são:
* `isin()`: checa se a coluna contém os valores listados na função.
* `contains()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Aceita uma outra coluna de texto.
* `like()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Funciona de forma similar ao "LIKE" do SQL.
* `rlike()`: utilizado para verificar se uma coluna de texto contém algum padrão especificado (**aceita regex**). Funciona de forma similar ao "RLIKE" do SQL.
* `startswith()`: utilizado para verificar se uma coluna de texto começa com algum padrão especificado (**aceita regex**).
* `endswith()`: utilizado para verificar se uma coluna de texto termina com algum padrão especificado (**aceita regex**).
* `between()`: checa se os valores da coluna estão dentro do intervalo especificado. Os dois lados do intervalo são inclusivos.
* `isNull()`: retorna True se o valor da coluna é nulo
* `isNotNull()`: retorna True se o valor da coluna não é nulo

Outros métodos úteis:
* `alias()/name()`: usado para renomear as colunas em operações como select() e agg()
* `astype()/cast()`: usado para mudar o tipo das colunas. Aceita tanto um string como um tipo especificado pelo módulo pyspark.sql.types
* `substr()`: utilizado para cortar um string com base em índices dos caracteres 

In [ ]:
(
    df_titles
    .filter(col('primaryTitle').like('Avengers%'))
    .filter(col('titleType') == 'movie')
    .show()
)

In [ ]:
(
    df_titles
    .withColumn('startYear', col("startYear").cast('int'))
    .filter('startYear is not null')
    .show()
)

### Ordenando o DataFrame

A ordenação do DataFrame pode ser feita utilizando as funções `orderBy()` ou `sort()`. Algumas funções auxiliares importante para serem usadas ao ordenar:
* `asc()`: ordena a coluna de forma ascendente (default)
* `desc()`ordena a coluna de forma decrescente
* `asc_nulls_first() / desc_nulls_first()`: ordena a coluna de forma ascendente e decrescente, respectivamente, mantendo os campos nulos primeiro
* `asc_nulls_last() / desc_nulls_last()`: ordena a coluna de forma ascendente e decrescente, respectivamente, mantendo os campos nulos por último

In [ ]:
df_titles.show(5)

In [ ]:
from pyspark.sql.functions import desc

(
    df_titles
    .withColumn('startYear', col('startYear').cast('int'))
    .orderBy('startYear')
    .filter('titleType = "movie"')
    .show()
)

In [ ]:
from pyspark.sql.functions import desc_nulls_first #desc_nulls_last

(
    df_titles
    .withColumn('startYear', col('startYear').cast('int'))
    .orderBy(desc_nulls_first('startYear'))
    .show()
)

### Renomeando Colunas

Para renomear colunas, é utilizada a função `withColumnRenamed()`, da seguinte forma:

```
df.withColumnRenamed("nome_antigo", "nome_novo")
```

In [ ]:
(
    df_titles
    .withColumnRenamed('primaryTitle', 'nome_filme')
    .show(5)
)

In [ ]:
(
    df_titles
    .withColumnRenamed('primaryTitle', 'nome_filme')
    .selectExpr('*', 'runtimeMinutes + 1')   
    .limit(5)
    .toPandas()
)

In [ ]:
(
    df_titles.selectExpr('primaryTitle as nome_filme', 'titleType', 'startYear', 'runtimeMinutes')
    .show(5)
)

In [ ]:
df_renamed = df_titles
for c in df_titles.columns:
    df_renamed = df_renamed.withColumnRenamed(c, c + '_suffix')

df_renamed.limit(5).toPandas()

### Criando e Alterando Colunas

Para criar ou alterar colunas, é utilizada a função `withColumn()`, da seguinte forma:

```
df.withColumn("nome_da_coluna", {expressão geradora de coluna})
```

In [ ]:
from pyspark.sql.functions import upper

(
    df_titles
    .select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn("primaryTitle_2", upper('primaryTitle'))
    .show(5)
)

#### Criando colunas a partir de constantes

In [ ]:
from pyspark.sql.functions import lit

(
    df_titles
    .select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn("pais", lit('Brasil'))
    .show(5)
)

#### Criando colunas condicionais

In [ ]:
from pyspark.sql.functions import when, expr

predicado = """

CASE WHEN runTimeMinutes <= 60 THEN 'curto'
     WHEN runTimeMinutes > 60 AND runTimeMinutes < 120 THEN 'normal'
     WHEN runTimeMinutes >= 120 THEN 'longo'
     WHEN runTimeMinutes IS NULL THEN 'nulo'
     ELSE 'Erro'
END

"""

(
    df_titles
    .select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn("runtimeMinutes", col('runTimeMinutes').cast('int'))
    .withColumn("categoria_runtime", expr(predicado))
    .filter('runTimeMinutes > 60')
    .show(25)
)

## Trabalhando com Diferentes Tipos de Dados

In [ ]:
import pyspark.sql.functions as f

### Valores Numéricos

* `round()`: arredonda o valor numérico
* `ceil()`: arredonda o valor numérico para o maior inteiro mais próximo
* `floor()`: arredonda o valor numérico para o menor inteiro mais próximo
* `sqrt()`: retorna a raiz quadrada do valor
* `exp()`: retorna a exponencial do valor
* `log()`: retorna a logaritmo natural do valor
* `log10()`: retorna a logaritmo na base 10 do valor
* `greatest()`: retorna o maior valor dentre os valores das colunas. Análogo ao `max()`, mas entre colunas
* `least()`: retorna o menor valor dentre os valores das colunas. Análogo ao `min()`, mas entre colunas

In [ ]:
df_titles.show(5)

In [ ]:
(
    df_titles
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .withColumn('random_normal', f.randn(123))
    .withColumn('dummy_division', f.col('runtimeMinutes') / f.col('random_normal'))
    .withColumn('round_example', f.round(f.col('dummy_division'), 3))
    .withColumn('ceil_example', f.ceil(f.col('dummy_division')))
    .withColumn('floor_example', f.floor(f.col('dummy_division')))
    .withColumn('greatest_example', f.greatest(f.col('random_normal'), f.col('runtimeMinutes'), f.lit(15)))
    .withColumn('least_example', f.least(f.col('random_normal'), f.col('runtimeMinutes'), f.lit(-15)))
    .limit(5)
    .toPandas()
)

### Strings

* `upper()`: retorna o string em letras maiúsculas
* `lower()`: retorna o string em letras minúsculas
* `initcap()`: retorna a primeira letra de cada palavra no string em letras maiúsculas
* `trim()`: retira os espaços em branco do início e do final do string
* `ltrim() / rtrim()`: retira os espaços em branco do início e do final do string, respectivamente
* `lpad() / rpad()`: acrescenta um caractere no início e no final do string, respectivamente, até que o string tenha um determinado comprimento
* `length()`: retorna o comprimento do string, em quantidade de caracteres
* `split()`: quebra o string a partir de um padrão e retorna um array com os string resultantes
* `concat()`: concatena uma ou mais colunas de string
* `concat_ws()`: concatena uma ou mais colunas de string, com um separador entre elas
* `regexp_extract()`: retorna um match no string a partir de um padrão regex
* `regexp_replace()`: substitui um mtach no strinf a partir de um padrão regex com outros caracteres
* `substring()`: retorna os caracteres do string que estão entre dos indices especificados. Análogo a `f.col().substring()`

In [ ]:
(
    df_titles
    .withColumn('primaryTitle', f.initcap(f.col('primaryTitle')))
    .withColumn('titleType', f.trim(f.initcap(f.col('titleType'))))
    .withColumn('genres_array', f.split(f.col('genres'), ','))
    .withColumn('num_const', f.substring(f.col('tconst'), 3, 7))
    .withColumn('full_name', f.concat_ws(' / ', f.col('primaryTitle'), f.col('originalTitle')))
    .limit(5)
    .toPandas()
)

### Datas

* `add_months()`: retorna a data depois de adicionar "x" meses
* `months_between()`: retorna a diferença entre duas datas em meses
* `date_add()`: retorna a data depois de adicionar "x" dias
* `date_sub()`: retorna a data depois de subtrair "x" dias
* `next_day()`: retorna o dia seguinte de alguma data
* `datediff()`: retorna a diferença entre duas datas em dias
* `current_date()`: retorna a data atual
* `dayofweek() / dayofmonth() / dayofyear()`: retorna o dia relativo à semana, ao mês e ao ano, respectivamente
* `weekofyear()`: retorna a semana relativa ao ano
* `second() / minute() / hour()`: retorna os segundos, os minutos e as horas de uma coluna de date-time, respectivamente
* `month() / year()`: retorna o mês e o ano de uma coluna de data, respectivamente
* `last_day()`: retorna o último dia do mês do qual a data considerada pertence
* `to_date()`: transforma a coluna no tipo data (t.DateType())
* `trunc()`: formata a data para a unidade especificada
    * `year`: "{ano}-01-01"
    * `month`: "{ano}-{mes}-01"

In [ ]:
(
    df_titles
    .filter('titleType = "movie"')
    .withColumn('data_ano', f.to_date(f.col('startYear'), 'yyyy'))
    .withColumn('mes', f.month(f.col('data_ano')))
    .withColumn('dia', f.dayofmonth(f.col('data_ano')))
    .withColumn('hoje', f.current_date())
    .withColumn('data_mes', f.trunc(f.col('hoje'), 'month'))
    .withColumn('ultimo_dia_mes', f.last_day(f.col('data_ano')))
    .withColumn('idade_filme_dias', f.datediff(f.col('hoje'), f.col('data_ano')))
    .withColumn('idade_filme_meses', f.floor(f.months_between(f.col('hoje'), f.col('data_ano'))))
    .withColumn('idade_filme_anos', f.floor(f.col('idade_filme_dias') / 365))
    .limit(5)
    .toPandas()
)

### Arrays

* `array()`: constrói um array com as colunas selecionadas
* `flatten()`: transforma um array de arrays em um unico array
* `explode()`: retorna uma nova linha para cada elemento no array 
* `size()`: retorna o número de elementos no array
* `sort_array()`: ordena os elementos do array, de forma crescente ou decrescente
* `reverse()`: reverte a ordem dos elementos de um array
* `array_distinct()`: remove elementos duplicados do array
* `array_contains()`: verifica se o array contém o elemento especificado
* `arrays_overlap()`: partir de 2 colunas de arrays, verifica se elas tem algum elemento em comum, retornando True ou False
* `array_union()`: a partir de 2 colunas de arrays, retorna um array com os elementos unidos das duas colunas, sem duplicatas
* `array_except()`: a partir de 2 colunas de arrays, retorna um array com os elementos que estão em uma coluna mas não estão na outra, sem duplicatas
* `array_intersect()`: a partir de 2 colunas de arrays, retorna um array com os elementos que nas duas colunas, sem duplicatas
* `array_join()`: retorna um string após concatenar os elementos do array usando o delimitador especificado
* `array_max() / array_min()`: retorna o máximo e o mínimo valor do array, respectivamente
* `array_remove()`: remove todos os elementos do array que são iguais ao valor especificado


In [ ]:
(
    df_titles
    .filter('titleType = "movie"')
    .withColumn('genres_array', f.split(f.col('genres'), ','))
#     .withColumn('first_genre', f.col('genres_array')[0])
#      .withColumn('second_genre', f.col('genres_array').getItem(1))
#      .withColumn('genres_string', f.array_join(f.col('genres_array'), ','))
#      .withColumn('n_genres', f.size(f.col('genres_array')))
#      .filter('n_genres >= 3')
    .withColumn('genres_unico', f.explode(f.col('genres_array')))
    .limit(5)
    .toPandas()
)

### Nulos

* `drop()`: retira do DataFrame as linhas com nulos, com base no que foi passado para o argumento how
    * any (default): retira todas as linhas com pelo menos um valor nulo nas colunas
    * all: somente retira as linhas com todos os valores nulos nas colunas
* `fill()`: preenche os valores nulos no DataFrame com uma constante, passada pelo usuário
* `replace()`: substitui o valor (não somente os valores nulos) por algum outro passado pelo usuário



In [ ]:
(
    df_titles
    .replace('\\N', None, subset=['startYear', 'endYear', 'runtimeMinutes'])
    .filter("startYear is null and runtimeMinutes is not null")
#     .na.fill('Não se sabe', subset=['startYear'])
#     .orderBy(f.asc_nulls_first('endYear'))
#     .na.drop(subset=['startYear'])
    .withColumn('coalesce_test', f.coalesce(f.col("startYear"), f.col("runtimeMinutes"), f.lit('Sem ano')))
    .limit(5)
    .toPandas()
)

### Agregação e Agrupamento

O agrupamento dos DataFrames é feito por meio da função **`groupby()`**. Essa função deve ser sucedida pela função de agregação `agg()`, de pivotação `pivot()` ou `count()`. 

---

A função **`agg()`** aplica uma função de agregação no DataFrame. Se precedida por `groupby()`, realiza a agregação dentro dos grupos esabelecidos.
Algumas das funções de agregação mais comuns:
* `sum()`: retorna a soma os valores da coluna
* `sumDistinct()`: retorna a soma os valores distintos da coluna
* `max() / min()`: retorna o mínimo e o máximo da coluna, respectivamente
* `avg() / mean()`: retorna a média dos valores da coluna
* `percentile_approx()`: retorna o percentil da coluna, comaproximação. Para trazer a mediana exata, usar: `percentile_approx(f.col('column'), 0.5, lit(1000000))`
* `stddev()`: retorna o desvio padrão dos valores da coluna
* `count()`: retorna a contagem de linhas
* `countDistinct()`: retorna a contagem de valores distintos da coluna
* `first() / last()`: retorna o primeiro e o último valor da coluna no agrupamento, respectivamente. Interessante de ser utilizada em conjunto com o argumento `ignoreNulls=True`.
* `collect_list()`: retorna os valores do agrupamento em uma lista, com duplicações
* `collect_set()`: retorna os valores do agrupamento em uma lista, sem duplicações (desordenado)

**Obs**: O spark ignora os valores nulos para calcular as agregações, com exceção da função `count()`.

---

A função **`pivot`** é utilizada para passar valores de uma linha para as colunas, realizando uma agregação. Deve ser sucedido por uma função de agregação utilizando `agg()`. Pode utilizar qualquer uma das funções de agregação anteriores.



In [ ]:
df_titles_subset = (
    df_titles
    .filter("cast(startYear as int) >= 2000")
    .sample(fraction = 0.5)
    .withColumn('genre', f.split('genres', ',').getItem(0))
)

In [ ]:
df_titles_subset.limit(5).toPandas()

In [ ]:
(
    df_titles_subset
    .agg(f.countDistinct('genre').alias('distinct_genres'),)
    .toPandas()
)

In [ ]:
(
    df_titles_subset
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('date'))
    )
    .toPandas()
)

In [ ]:
df_titles_subset.describe().toPandas()

In [ ]:
(
    df_titles_subset
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .select('runtimeMinutes')
    .describe()
    .toPandas()
)

#### Agrupamento

In [ ]:
df_titles_subset.limit(5).toPandas()

In [ ]:
(
    df_titles_subset
    .groupby('genre', 'startYear')
    .agg(f.mean('runtimeMinutes').alias('mean_runtimeMinutes'),)
    .orderBy('startYear', f.col('mean_runtimeMinutes').desc())
    .filter('startYear = 2021')
    .toPandas()
)

In [ ]:
(
    df_titles_subset
    .groupby('genre')
    .agg(f.collect_set(f.col('titleType')).alias('lista_tipos_titulo'),
         f.countDistinct(f.col('titleType')).alias('n_distinct')
        )
    .withColumn('tipos_filmes', f.explode(f.col('lista_tipos_titulo')))
    .select('genre', 'tipos_filmes')
    .toPandas()
)

#### Pivotação

In [ ]:
df_titles_subset.limit(5).toPandas()

In [ ]:
(
    df_titles_subset
    .drop('genre')
    .withColumn('genres', f.explode(f.split(f.col('genres'), ',')))
    .groupby('startYear')
    .pivot('genres')
    .agg(f.mean('runtimeMinutes'))
    .na.fill(0)
    .orderBy('startYear')
    .limit(5)
    .toPandas()
)

### Window Functions

Window functions são funções que realizam cálculos similares à uma agregação, mas que não resultam em um DataFrame agregado. Ao invés disso, os resultados são colocados em uma nova coluna, segundo a partição (ou agrupamento) especificado. 
Exemplos mais comuns:
* `row_number()`
* `rank() / dense_rank() / percent_rank()`
* `lag()`
* `cume_dist()`
* `collect_list() / collect_set()`
* Demais funções de agregação, com exceção de `countDistinct()`

Para usar as funções dessa forma, devemos criar uma janela (window) da seguinte forma:

```{python}
from pyspark.sql.window import Window
w = Window.partitionBy({columns}).orderBy({columns}).rowsBetween({lower}, {upper})
```

* **`partitionBy()`**: agrupamento em que os cálculos serão realizados. É análogo ao `groupBy()`.
* **`orderBy`**: algumas funções como `row_number()` e `lag()` dependem da ordenação das linhas do agrupamento. Essa função é usada para especificar essa ordem.
* **`rowsBetween()`**: esse método é usado para especificar janelas deslizantes. A partir dele é possível definir um intervalo de linhas, relativas à linha atual, em que a função vai ser aplicada. Caso isso não seja especificado, as operações são realizadas em todo o grupo. Muito útil para construir **médias móveis**. Os seguintes objetos ajudam na constrção desse intervalo:
  * `Window.currentRow`: define a linha para qual o valor está sendo calculado como um dos limites de cálculo
  * `Window.unboundedPreceding`: define que não há limites para as linhas anteriores à linha para qual o valor está sendo calculado, isto é, a função irá considerar todas as linhas do grupo que já passaram. Deve ser usado no primeiro argumento (start).
  * `Window.unboundedFollowing`: define que não há limites para as linhas posteriores à linha para qual o valor está sendo calculado, isto é, a função irá considerar todas as linhas do grupo que ainda não passaram. Deve ser usado no segundo argumento (end).

Depois disso, basta utilizar a função `over()` para indicar que aquela função deve ser realizada na janela.  Exemplo:
```
df.withColumn('rn', f.row_number().over(w))
```


In [ ]:
from pyspark.sql.window import Window

In [ ]:
df_titles_subset = (
    df_titles
    .filter("cast(startYear as int) >= 2000")
    .sample(fraction = 0.5)
    .withColumn('genre', f.split('genres', ',').getItem(0))
)

In [ ]:
df_titles_subset.count()

In [ ]:
df_titles_subset.withColumn('genre', f.split('genres', ',').getItem(0)).limit(5).toPandas()

In [ ]:
w = Window.partitionBy('genre').orderBy(f.desc('startYear'))
(
    df_titles_subset
    .withColumn('genre', f.split('genres', ',').getItem(0))
    .withColumn('startYear', f.col('startYear').cast('int'))
    .filter('startYear >= 2021')
    .withColumn('rn', f.percent_rank().over(w))
    .limit(25)
    .toPandas()
)

In [ ]:
w = Window.partitionBy('titleType', 'startYear')
(
    df_titles_subset
    .withColumn('genre', f.split('genres', ',').getItem(0))
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .withColumn('total_minutes', f.sum(f.col('runtimeMinutes')).over(w))
    .withColumn('mean_minutes', f.mean(f.col('runtimeMinutes')).over(w))
    .withColumn('relative_minutes', f.col('runtimeMinutes') / f.col('total_minutes'))
    .filter('runtimeMinutes is not null') 
    .limit(5)
    .toPandas()
)

### Cálculo de média móvel

In [ ]:
w = Window.partitionBy('titleType').orderBy('startYear').rowsBetween(Window.unboundedPreceding, Window.currentRow)
(
    df_titles_subset
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .groupby('titleType', 'startYear')
    .agg(f.expr('mean(runtimeMinutes) as media_minutos'))
    .orderBy('titleType', 'startYear')
    .withColumn('meadia_movel_3anos', f.round(f.mean('media_minutos').over(w), 3))
    .limit(15)
    .toPandas()
)

In [ ]:
#f.round(((50.256494 + 51.645619 + 51.713004)/3), 3)

In [ ]:
#(49.186983 + 50.358881 + 52.182771 + 49.007634 + 55.584795)/5 == 51.264

#### Usando uma Window para calcular os distintos

In [ ]:
w = Window.partitionBy('titleType', 'startYear')
(
    df_titles
    .withColumn('lista_cnae', f.countDistinct(f.col('tconst')).over(w))
    .limit(5)
    .toPandas()
)

In [ ]:
w = Window.partitionBy('titleType', 'startYear')
(
    df_titles_subset
    .withColumn('lista_titulos', f.collect_set(f.col('tconst')).over(w))
    .limit(5)
    .toPandas()
)

In [ ]:
w = Window.partitionBy('titleType', 'startYear')
(
    df_titles_subset
    .withColumn('lista_titulos', f.collect_set(f.col('tconst')).over(w))
    .withColumn('titulos_distintos', f.size(f.col('lista_titulos')))
    .select('titleType', 'startYear', 'titulos_distintos')
    .orderBy(f.col('titulos_distintos').desc())
    .limit(5)
    .toPandas()
)

In [ ]:
(
    df_titles_subset
    .groupby('titleType', 'startYear')
    .agg(f.countDistinct(f.col('tconst')).alias('titulos_distintos'))
    .orderBy(f.col('titulos_distintos').desc())
    .limit(5)
    .toPandas()
)

#### Usando Windows para evitar Joins

Objetivo: Titulos mais recentes por genero

Caminho natural:
```
df1 = df_titles.groupby('genre').agg(f.max(f.col('startYear').alias('startYear'))
df2 = df_titles.join(df1, ['genre', 'startYear'])
```
Alternativa:

In [ ]:
w = Window.partitionBy('genre')
(
    df_titles_subset
    .withColumn('max_data', f.max(f.col('startYear')).over(w))
    .filter('startYear = max_data')
    .limit(10)
    .toPandas()
)

### Joins

Os joins no pyspark são especificados pela função `join()`, da seguinte forma:

```
df1.join(df2, {key_columns}, {join_type})
```

* `key_columns`: colunas que vão ser utilizadas para fazer a junção das tabelas. Pode ser especificada como
    * Um único string -> só uma coluna é chave, mesmos nomes nas duas tabelas
    * Uma lista de string ou de colunas (`col()`) -> mais de uma coluna é chave, mesmos nomes nas duas tabelas
    * Com nomes diferentes, é necessário fazer uma especificação do tipo: `f.col(column1) == f.col(column2)`. Caso existam mais de uma coluna como chave, essas especificações devem ser colocadas em uma lista.
* `join_type`: o tipo de join a ser realizado. As opções são:
    * `inner (default)`: INNER JOIN do SQL
    * `outer / full / fullouter / full_outer`: : FULL OUTER JOIN do SQL
    * `left / leftouter / left_outer`: : LEFT JOIN do SQL
    * `right / rightouter / right_outer`: : RIGHT JOIN do SQL
    * `semi / leftsemi / left_semi`: realiza um LEFT JOIN do SQL e retorna somente as colunas do DataFrame da esquerda que também estão no DataFrame da Direita
    * `anti / leftanti / left_anti`: realiza um LEFT JOIN do SQL e retorna somente as colunas do DataFrame da esquerda que não estão no DataFrame da Direita

In [1]:

df_ratings = (spark.read
    .format('csv')
    .options(sep='\t', header=True)
    .load(imdb_path + 'title_ratings.tsv')
)

NameError: name 'spark' is not defined

In [ ]:
df_ratings.limit(5).toPandas()

In [ ]:
df_titles.limit(5).toPandas()

In [ ]:
df_ratings.count()

In [ ]:
df_titles.count()

In [ ]:
(
    df_titles
    .join(df_ratings, 'tconst', 'left')
    .filter('averageRating is null')
    .limit(5)
    .toPandas()
)

In [ ]:
(
    df_titles
    .withColumnRenamed('tconst', 'id_title')
    .join(df_ratings, f.col('tconst') == f.col('id_title'))
    .withColumn('averageRating', f.expr('averageRating + 1'))
    .limit(5)
    .toPandas()
)

#### Utilizando semi e anti join

In [ ]:
df_ratings.select('tconst').distinct().count()

In [ ]:
df_titles.select('tconst').distinct().count()

In [ ]:
(
    df_titles
    .join(df_ratings, 'tconst', 'semi')
    .count()
)

In [ ]:
(
    df_titles
    .join(df_ratings, 'tconst', 'anti')
    .count()
)

In [ ]:
6961705 + 1174232 == df_titles.count()

### Union

Existem três formas de unir DataFrames no pyspark:
* `union() / unionAll()`: empilha os DataFrames, preservando linhas duplicadas. As colunas são unidas por posição, e por isso a ordem delas deve ser a mesma entre os dois DFs.
* `unionByName()`: empilha os DataFrames, preservando linhas duplicadas. As colunas são unidas por nome, e por tanto não precisam estar ordenadas da mesma forma

In [ ]:
df_titles.limit(5).toPandas()

In [ ]:
df_titles.count()

In [ ]:
df1 = df_titles.sample(fraction = 0.5)
df2 = df_titles.join(df1, ['tconst'], 'anti')

In [ ]:
df1.count()

In [ ]:
df2.count()

In [ ]:
df1.union(df2).count()

In [ ]:
df3 = df_titles.sample(fraction = 0.05)

In [ ]:
df3.count()

In [ ]:
df3.union(df3).count()

In [ ]:
df3.union(df3).distinct().count()

In [ ]:
df2 = df2.select(df2.columns[::-1])

In [ ]:
df1.limit(5).toPandas()

In [ ]:
df2.limit(5).toPandas()

In [ ]:
df1.union(df2).filter('genres rlike "[0-9]"').limit(5).toPandas()

In [ ]:
df1.unionByName(df2).filter('genres rlike "[0-9]"').limit(5).toPandas()

### User Defined Functions (UDFs)

Em algumas situações é necessário criar/alterar uma coluna utilizando uma operação não implementada na biblioteca padrão. Para isso, é possível utilzar funções definidas pelo usuário (UDFs) por meio da função `udf()`.

**Importante**: As udfs não são otimizadas para serem executadas em paralelo, e por isso podem representar um gargalo na na aplicação.

In [9]:
from unidecode import unidecode
from pyspark.sql.types import StringType

In [10]:
unidecode('àáâçéõü')

'aaaceou'

In [ ]:
def unidecode_function(string):
    if not string:
        return None
    else:
        return unidecode(string)

unidecode_udf = f.udf(unidecode_function, returnType=StringType())

In [ ]:
(
    df_titles
    .filter(f.col('primaryTitle').rlike('à|á|â|ç|é|õ|ü|ó'))
    .withColumn('cleaned_string', unidecode_udf(f.col('primaryTitle')))
    .select('primaryTitle', 'cleaned_string')
    .limit(5)
    .toPandas()
)

In [ ]:
del unidecode_udf

In [ ]:
@f.udf(returnType=t.StringType())
def unidecode_udf(string):
    if not string:
        return None
    else:
        return unidecode(string)

In [ ]:
(
    df_titles
    .filter(f.col('primaryTitle').rlike('à|á|â|ç|é|õ|ü'))
    .withColumn('cleaned_string', unidecode_udf(f.expr('primaryTitle')))
    .select('primaryTitle', 'cleaned_string')
    .limit(5)
    .toPandas()
)

### Criando Métodos Customizado

Em algumas situações, é interessante que realizemos uma operação sobre um DataFrame que não está implementada. Além disso, pode ser que seja necessário (ou do desejo do desenvolvedor) utilizar essa operação de forma encadeada.

Para resolver esse problema, podemos utilizar o método `.transform()`. Funciona da seguinte maneira:

1) Definir uma função do python da seguinte forma:

```
def f(args):
  def _(df):
    {operacoes sob o DataFrame}
    return df
  return _
```
2) Depois de definida a função, ela pode ser chamada da seguinte forma:


```
df.transform(f(args))
```

In [ ]:
def processing(df):
    ...
    return df

df = (
    df.select().filter()
)
df = processing(df)
df = (
    df.groupby().agg()
)

df = (
    df.select()
    .filter()
    .transform(processing(df))
    .groupby()
    .agg()
)

In [ ]:
def renamer(dict):
    def _(df):
        for c, n in dict.items():
            df = df.withColumnRenamed(c, n)
        return df
    return _

In [ ]:
df_titles.limit(5).toPandas()

In [ ]:
rename_dict = {
    "tconst": 'id_title',
    "titleType": 'tipo_title',
    'primaryTitle': 'nome_primario',
    'originalTitle': 'nome_original',
    'isAdult': "idc_adult_title",
    'startYear': 'ano_lancamento',
    'endYear': 'ano_encerramento',
    'runtimeMinutes': 'duracao_minutos',
    'genres': 'generos',
}

(
    df_titles
    .transform(renamer(rename_dict))
    .limit(5)
    .toPandas()
)

In [ ]:
from pyspark.sql import DataFrame

def transform(self, f):
    return f(self)

DataFrame.transform = transform